In [ ]:
MutationFile = 'Cutaneous_somatic_mutation_MELA_AU_DNVs_sorted_dinuc_filter.bed'
FastaFile = 'hg19.fa'
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'

from SequenceContext import CreateNewBedFile
CreateNewBedFile(1,1,MutationFile,FastaFile)

from CompareMutationContext import AttainMutationData
AttainMutationData('AC>TT', OutputFile)



In [ ]:
MutationFile = 'Cutaneous_somatic_mutation_MELA_AU_DNVs_sorted_dinuc_filter.bed'
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'
FastaFile = 'hg19.fa'
from ActuallyNormalizeData import NormalizeData
NormalizeData(OutputFile, FastaFile, 4)


In [ ]:
# Bar graphs
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'
gdata = pd.read_csv('Normalized_'+OutputFile)
gdata = gdata.sort_values('normalizedCounts', ascending = False)
sns.set_theme(style = 'whitegrid')
sns.barplot(y ='sequenceContext', x = 'normalizedCounts', data = gdata, color = 'red')

In [ ]:
OutputFile = 'AC>TT_Mutation_Context_Frequency.csv'
from CreateLinearRegressionFile import LinearFile
from PearsonR import RegressionAnalysis
LinearFile(OutputFile)
RegressionAnalysis(OutputFile)